In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os
import shutil
import time
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import decode_predictions

# Chargement des données et division en train, val et test  

In [17]:
df=pd.read_csv('X_train_update.csv')
df1=pd.read_csv("y_train_CVw08PX.csv")


#Nous allons changer les valeurs de notre colonne chemin_images du dataFrame par les chemins des images correspondantes à productid et imageid
for mmm, mmm1 in zip(df.imageid, df.productid):    
        nom_img = f"image_{mmm}_product_{mmm1}.jpg"
        df.loc[df['imageid'] == mmm, 'image_name'] = nom_img
 
#On peut supprimer les colonnes productid et imageid de notre data Frame enfin de l'alleger.
df=df.drop(["designation","productid", "imageid","description"], axis=1)

In [18]:
df["categ"] = df1.prdtypecode
df.head()

,Unnamed: 0,image_name,categ
0,0,image_1263597046_product_3804725264.jpg,10
1,1,image_1008141237_product_436067568.jpg,2280
2,2,image_938777978_product_201115110.jpg,50
3,3,image_457047496_product_50418756.jpg,1280
4,4,image_1077757786_product_278535884.jpg,2705


In [4]:
# Chemins
CSV_PATH = './labels.csv'
IMG_DIR = 'C:/Users/Bureau/Desktop/projetRakuten/images/train/image_train'
DEST_DIR = 'C:/Users/Bureau/Desktop/projetRakuten/images/train/dataset_split'  # dossier de destination

# Lire le CSV
#df = pd.read_csv(CSV_PATH)

# Diviser en train (70%) / val (15%) / test (15%)
df_train, df_temp = train_test_split(df, test_size=0.3, stratify=df['categ'], random_state=42)
df_val, df_test = train_test_split(df_temp, test_size=0.5, stratify=df_temp['categ'], random_state=42)

 Fonction pour copier les images dans un dossier trié
def save_split(df_split, split_name):
    for _, row in df_split.iterrows():
        try:
            label = row['categ']
            filename = row['image_name']
        except KeyError:
            print("Erreur : colonne 'label' ou 'image_name' introuvable.")
            print(f"Colonnes disponibles : {df_split.columns}")
            break  # ou continue selon ton besoin
        src_path = os.path.join(IMG_DIR, filename)
        dest_path = os.path.join(DEST_DIR, split_name, str(label))
        os.makedirs(dest_path, exist_ok=True)
        try:
            shutil.copy(src_path, os.path.join(dest_path, filename))
        except FileNotFoundError:
            print(f"Image not found: {filename}")
            
# Copier les fichiers
save_split(df_train, 'train')
save_split(df_val, 'val')
save_split(df_test, 'test')

# Création de modèle 

In [19]:
# Chemins
base_dir = 'C:/Users/Bureau/Desktop/projetRakuten/images/train/dataset_split'
img_size = (224, 224)
batch_size = 32

# Générateurs de données
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_data = train_gen.flow_from_directory(
    os.path.join(base_dir, 'train'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_gen.flow_from_directory(
    os.path.join(base_dir, 'val'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_data = test_gen.flow_from_directory(
    os.path.join(base_dir, 'test'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)


Found 59441 images belonging to 27 classes.
Found 12737 images belonging to 27 classes.
Found 12738 images belonging to 27 classes.


In [7]:
# Charger ResNet50 sans le top
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Ne pas entraîner les couches du modèle de base
for layer in base_model.layers:
    layer.trainable = False

# Ajouter les couches de classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(train_data.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compiler
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [15]:
# Entraînement
model.fit(
    train_data,
    validation_data=val_data,
    epochs=20
)

# Prédictions
y_true = test_data.classes  
y_pred_probs = model.predict(test_data)
y_pred = np.argmax(y_pred_probs, axis=1)

# Évaluation sur le test
loss, acc = model.evaluate(test_data)
print(f"Test accuracy: {acc:.2%}")
# F1-score macro
f1 = f1_score(y_true, y_pred, average='macro')
print(f"F1-score (macro) sur les données de validation : {f1:.4f}")

# Sauvegarde du modèle
model.save('resnet50_rakuten1.h5')


Epoch 1/20
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 1933s 1s/step - accuracy: 0.9768 - loss: 0.1408 - val_accuracy: 0.6205 - val_loss: 1.6502
Epoch 2/20
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 1919s 1s/step - accuracy: 0.9791 - loss: 0.1277 - val_accuracy: 0.6217 - val_loss: 1.6680
Epoch 3/20
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 1919s 1s/step - accuracy: 0.9838 - loss: 0.1097 - val_accuracy: 0.6188 - val_loss: 1.6876
Epoch 4/20
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 1915s 1s/step - accuracy: 0.9861 - loss: 0.0987 - val_accuracy: 0.6157 - val_loss: 1.7587
Epoch 5/20
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 1917s 1s/step - accuracy: 0.9876 - loss: 0.0892 - val_accuracy: 0.6139 - val_loss: 1.7631
Epoch 6/20
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 1918s 1s/step - accuracy: 0.9893 - loss: 0.0791 - val_accuracy: 0.6189 - val_loss: 1.8214
Epoch 7/20
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 1922s 1s/step - accuracy: 0.9895 - loss: 0.0751 - val_accuracy: 0.6220 - val_loss: 1.8307
Epoch 8/20
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 1920s 1s/step - accuracy: 0.9902 -

Test accuracy: 61.30%
F1-score (macro) sur les données de validation : 0.5651


## Calcul du F1-score moyen (macro, micro ou weighted selon le besoin)

In [20]:
MODEL_PATH = 'resnet50_rakuten1.h5'
model1 = load_model(MODEL_PATH)

In [21]:
# Prédictions
y_true = test_data.classes  # les vraies classes (entiers)
y_pred_probs = model1.predict(test_data)
y_pred = np.argmax(y_pred_probs, axis=1)

# Évaluation sur le test
loss, acc = model1.evaluate(test_data)
print(f"Test accuracy: {acc:.2%}")
# F1-score :
for f1_sc in ["macro", "micro", "weighted"]:
    f1 = f1_score(y_true, y_pred, average=f1_sc)  
    print(f"F1-score moyen {f1_sc}:", f1)

C:\Users\Bureau\Desktop\projetRakuten\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


399/399 ━━━━━━━━━━━━━━━━━━━━ 558s 1s/step
399/399 ━━━━━━━━━━━━━━━━━━━━ 563s 1s/step - accuracy: 0.5987 - loss: 2.2730
Test accuracy: 61.30%
F1-score moyen macro: 0.5650510634519045
F1-score moyen micro: 0.6130475741874706
F1-score moyen weighted: 0.6087380352752294
